In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys

from importlib import reload


sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

In [2]:
df_delisted = ut.concatenate_excel_sheets('./data/tej_財報_2000_2023_下市.xlsx')

In [3]:
df_listed_IFRS = ut.concatenate_excel_sheets('./data/tej_IFRS_2005_2023_上市.xlsx')

In [4]:
df_listed_Finance_DB = ut.concatenate_excel_sheets('./data/tej_FinanceDB_2000_2019_上市.xlsx')

In [62]:
df_listed_IFRS_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_cum_2005_2023_上市.xlsx')

In [70]:
df_delisted_lsd = ut.concatenate_excel_sheets('./data/tej_長短期借款_2000_2023_下市.xlsx')
df_delisted_lsd = ut.tej_data_clean_up(df_delisted_lsd)
df_delisted_lsd.drop_duplicates(inplace=True)

In [64]:
df_delisted = ut.tej_data_clean_up(df_delisted)
df_listed_IFRS = ut.tej_data_clean_up(df_listed_IFRS)
df_listed_Finance_DB = ut.tej_data_clean_up(df_listed_Finance_DB)
df_listed_IFRS_cum = ut.tej_data_clean_up(df_listed_IFRS_cum)

column is already exist
column is already exist
column is already exist


In [65]:
df_delisted.drop_duplicates(inplace=True)
df_listed_IFRS.drop_duplicates(inplace=True)
df_listed_Finance_DB.drop_duplicates(inplace=True)
df_listed_IFRS_cum.drop_duplicates(inplace=True)

df_delisted.columns = [col[1:].strip() if col[0] == 'Q' else col.strip() for col in df_delisted.columns]
df_listed_IFRS.columns = [col.strip() for col in df_listed_IFRS.columns]
df_listed_Finance_DB.columns = [col[1:].strip() if col[0] == 'M' else col.strip() for col in df_listed_Finance_DB.columns]

In [66]:
# to feather
df_delisted.reset_index(drop=True).to_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS.reset_index(drop=True).to_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB.reset_index(drop=True).to_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum.reset_index(drop=True).to_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')

In [105]:
# from feather
df_delisted = pd.read_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS = pd.read_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB = pd.read_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum = pd.read_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')



In [106]:
df_delisted = df_delisted.merge(df_delisted_lsd[['股票代號','年月','長短期借款']], on=['股票代號', '年月'], how='left')
df_listed_Finance_DB['長短期借款'] = np.nan

In [109]:
df_delisted_columns = pd.DataFrame(df_delisted.columns, columns=['delisted'])
df_listed_IFRS_columns = pd.DataFrame(df_listed_IFRS.columns, columns=['IFRS'])
df_listed_Finance_DB_columns = pd.DataFrame(df_listed_Finance_DB.columns, columns=['Finance_DB'])

# merge all df to check the columns
df_all_columns = pd.merge(df_delisted_columns, df_listed_IFRS_columns, how='outer', left_on='delisted', right_on='IFRS')
df_all_columns = pd.merge(df_all_columns, df_listed_Finance_DB_columns, how='outer', left_on='delisted', right_on='Finance_DB')

In [111]:
# drop the columns that only Finance_DB has na
df_all_columns = df_all_columns.dropna(how = 'any')

In [134]:
df_listed_Finance_DB['sequence'] = 1
df_listed_IFRS['sequence'] = 2
df_delisted['sequence'] = 3

In [ ]:
common_column = df_all_columns['delisted'].tolist()

In [143]:
df_fs_all = pd.DataFrame([])
# concat the all df
df_fs_all = pd.concat([df_fs_all, df_listed_IFRS[common_column + ['sequence']]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_listed_Finance_DB[common_column + ['sequence']]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_delisted[common_column + ['sequence']]],axis=0)

In [148]:
df_seq = df_fs_all.groupby(['股票代號','年月'])['sequence'].min().reset_index().copy()

In [149]:
df_fs_merge = pd.merge(df_seq, df_fs_all, how='left', on=['股票代號','年月','sequence'])

In [153]:
df_fs_merge.sort_values(['股票代號','年月'], inplace=True)

In [156]:
# filter out the data that '股票代號' length is not 4
df_fs_merge = df_fs_merge[df_fs_merge['股票代號'].str.len() == 4]

In [159]:
df_fs_merge.reset_index(drop=True, inplace=True)

In [160]:
df_fs_merge.to_feather('./data/df_tej_fs_merge.feather')